# Classification, Regression and Other Prediction Model

## Dataset

We‘ll use "201707-citibike-tripdata.csv.zip" (after preprocessed in HW0)

## Schema

- Every station’s information
    - id, name, lat, lng
- Every stations’ flow data
    - id, time, in-flow, out-flow

### Import packages

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import os
from time import time
from plotly.graph_objs import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeClassifier
from sklearn.linear_model import BayesianRidge
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

### Read csv to dataframe
use pandas to read data

In [14]:
# preprocessed dataset
df = pd.read_csv('./201707-citibike-tripdata-preprocessed.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2


In [15]:
# every station's information
station_info = pd.read_csv('./station_info.csv')
station_info.head()

,station id,station name,station latitude,station logitude
0,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241
1,293,Lafayette St & E 8 St,40.730207,-73.991026
2,3242,Schermerhorn St & Court St,40.691029,-73.991834
3,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198
4,361,Allen St & Hester St,40.716059,-73.991908


In [16]:
# every station's in-flow data
station_in_flow = pd.read_csv('./in_flow.csv')
station_in_flow.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,1.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0
2,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# every station's out-flow data
station_out_flow = pd.read_csv('./out_flow.csv')
station_out_flow.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Using historical (14 days) data to predict every station's outflow tomorrow (1 day)

### Extract following values

- station_id
- outflow(and this is we want to predict)

In [18]:
station_out_flow.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Discretize outflow
- discretize with divided by every station's outflow standard deviation and round to integer
- process them so it can be solve as a classification problem

By previous homework's results, we can find divided by 5 is a good way to discretize so apply it and round the value to integer.

In [19]:
station_out_dis = (station_out_flow / 5).round(0)
station_out_dis.head()

,72,79,82,83,116,119,120,127,128,143,...,2003,2005,2006,2008,2009,2010,2012,2021,2022,2023
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Use previous (14 days) data to estimate next days’ outflow
- use a sliding window to increase our data (shift k days each time, and determine the k = 1 )

In [20]:
def get_data(isdis, idx, st):
    if isdis == True:
        df = pd.DataFrame(station_out_dis.iloc[st * 48 : (15 + st) * 48, idx]).T
    else:
        df = pd.DataFrame(station_out_flow.iloc[st * 48 : (15 + st) * 48, idx]).T
    df.columns = [i for i in range(df.shape[1])]
    return df

def get_station(isdis, idx):
    data = pd.DataFrame()
    res = []
    for i in range(16):
        data = data.append(get_data(isdis, idx, i))
    return data

- We can use ```get_station(is_discrete, index)``` to get the station's outflow data from 7/01 - 7/15 to 7/16 - 7/30 in each row

In [21]:
get_station(True, 1).head()

,0,1,2,3,4,5,6,7,8,9,...,710,711,712,713,714,715,716,717,718,719
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Evaluate each model

Calculate the mean accuracy, mean square error and using time

In [22]:
def eval_model(isdis, clf):
    ans = 0
    t = time()
    for idx in range(634):
        data = get_station(isdis, idx)
        train_x, test_x, train_y, test_y = train_test_split(data.iloc[:, :14 * 48], data.iloc[:, 14 * 48:], test_size = 0.3)
        for i in range(48):
            ans += clf.fit(train_x, train_y.iloc[:, i]).score(test_x, test_y.iloc[:, i])
    if isdis == True:
        print 'Average accuracy for 48 timeslot: {:.4f}'.format((ans / 634.0 / 48.0))
    else:
        print 'Mean square error for 48 timeslot: {:.4f}'.format((ans / 634.0 / 48.0))
    print 'Time: {:.2f} sec'.format(time() - t)

## Try following models (as classification problem)

compare the computation time and result ( average accuracy for 48 timeslot )

### K-Nearest-Neighbor

Classifier implementing the k-nearest neighbors vote.

By previous homework, the results of Kmeans and PCA => Agglomerative Clustering look like we can divided the data into 3 - 4 parts so we choose k = 3 or 4.

[package](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [23]:
clf = OneVsRestClassifier(KNeighborsClassifier(n_neighbors = 3))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7748
Time: 104.26 sec


In [24]:
clf = OneVsRestClassifier(KNeighborsClassifier(n_neighbors = 4))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7897
Time: 108.04 sec


### Naive Bayes

Try multinomial and Gaussian to predict the data.

- Naive Bayes classifier for multinomial models

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

- Gaussian Naive Bayes (GaussianNB)

Can perform online updates to model parameters via partial_fit method. For details on algorithm used to update feature means and variance online, see Stanford CS tech report STAN-CS-79-773 by Chan, Golub, and LeVeque:
http://i.stanford.edu/pub/cstr/reports/cs/tr/79/773/CS-TR-79-773.pdf

[package](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [25]:
clf = OneVsRestClassifier(MultinomialNB())
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7918
Time: 93.21 sec


In [26]:
clf = OneVsRestClassifier(GaussianNB())
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7696
Time: 99.21 sec


### Random Forest

That is a random forest classifier and setting max_depth to prevent the decision tree being too deep leads to overfitting and wasting time.

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

[package](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [27]:
clf = OneVsRestClassifier(RandomForestClassifier(max_depth = 2))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7866
Time: 771.42 sec


In [28]:
clf = OneVsRestClassifier(RandomForestClassifier(max_depth = 5))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7858
Time: 778.11 sec


### Support vector machine(SVC)

C-Support Vector Classification and the implementation is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to dataset with more than a couple of 10000 samples.

Try to use different kernels to see the accuracy and using time.

[package](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [46]:
ker = ['linear', 'poly', 'rbf', 'sigmoid']

for item in ker:
    print 'kernel: {}'.format(item)
    clf = OneVsRestClassifier(SVC(kernel = item))
    eval_model(True, clf)

kernel: linear
Average accuracy for 48 timeslot: 0.7944
Time: 131.80 sec
kernel: poly
Average accuracy for 48 timeslot: 0.7947
Time: 130.30 sec
kernel: rbf
Average accuracy for 48 timeslot: 0.7944
Time: 130.57 sec
kernel: sigmoid
Average accuracy for 48 timeslot: 0.7939
Time: 134.99 sec


### Other

Use extremely randomized tree classifier to predict the data.

Extra-trees differ from classic decision trees in the way they are built. When looking for the best split to separate the samples of a node into two groups, random splits are drawn for each of the max_features randomly selected features and the best split among those is chosen. When max_features is set 1, this amounts to building a totally random decision tree.

Also setting max_depth to prevent the decision tree being too deep leads to overfitting and wasting time.

[package](http://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeClassifier.html)

In [30]:
clf = OneVsRestClassifier(ExtraTreeClassifier(max_depth = 2))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7420
Time: 110.33 sec


In [39]:
clf = OneVsRestClassifier(ExtraTreeClassifier(max_depth = 5))
eval_model(True, clf)

Average accuracy for 48 timeslot: 0.7291
Time: 115.92 sec


### Compare and Observation

- K-Nearest-Neighbor

n_neighbors is set to 4 is more accuracy but the time it takes is also more. They take about 105 seconds and the accuracy is almost 80%.

- Naive Bayes

Choose the multinomial naive bayes is better because the outflows are match the multinomial model rather than Gaussian during weekday and weekend. The time they take is less than KNN and Multinomial's accuracy is almost 80% too.

- Random Forest

Compared with the max_depth and I find the time is getting much without improving accuracy. The time is much more than other models because it builds many trees to decide the predicted results.

- Support vector machine(SVC)

Every kernel's results are almost the same but sometime we can find accuracy is a little bit higher and taking less time in linear kernel.

- Extremely Randomized Tree

Extremely randomized tree classifier's accuracy is less than other models and the using time is not decreasing. QQ


## Calculate the confusion matrix

generate the label by collecting all the target values and construct the confusion matrix.

In [32]:
label = set()

for i in range(48):
    for item in pd.unique(station_out_dis.iloc[:, -i]):
        label.add(item)
label = list(label)
num_l = len(label)

clf = OneVsRestClassifier(MultinomialNB())
mat = np.zeros([num_l, num_l], dtype = np.int)
for idx in range(634):
        data = get_station(True, idx)
        train_x, test_x, train_y, test_y = train_test_split(data.iloc[:, :14 * 48], data.iloc[:, 14 * 48:], test_size = 0.3)
        mat += (confusion_matrix(clf.fit(train_x, train_y.iloc[:, 0]).predict(test_x), test_y.iloc[:, 0], labels = label))

Print the confusion matrix for predicting the first hour in one day
for Naive Bayes.

In [33]:
mat

array([[2984,  142,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [  30,   14,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],

## Performance with different parameters in SVM

Test following parameters

- kernel
    - linear
    - poly
    - rbf
    - sigmoid

The 

We can find the highest accuracy in rbf kernel and I think the reason is rbf can approximate of any non-linear function in  high precision. So it could be more match than the others and from previos work we know the data is more close to mutinormal distribution.

## Try following models (as regression problem)

compare the computation time and result ( Mean square error )

### ARIMA

In [34]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from statsmodels.tsa.arima_model import ARIMA

# Dataset
wpi1 = requests.get('http://www.stata-press.com/data/r12/wpi1.dta').content
data = pd.read_stata(BytesIO(wpi1))
data.index = data.t

print type(data['wpi'])


# mod = ARIMA(get_station(1).iloc[:, 0], order=(1,1,1))

# # Fit the model
# res = mod.fit(disp=False)
# print(res.summary())

<class 'pandas.core.series.Series'>


### Bayesian regression

Use Bayesian ridge regression and try to set n_iter to see how much time it takes and how accuracy it inprove.

Fit a Bayesian ridge model and optimize the regularization parameters lambda (precision of the weights) and alpha (precision of the noise).

[package](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html)

In [35]:
clf = OneVsRestClassifier(BayesianRidge(n_iter = 300))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.4756
Time: 667.29 sec


In [40]:
clf = OneVsRestClassifier(BayesianRidge(n_iter = 500))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.4744
Time: 613.90 sec


### Decision tree regression

A decision tree but use a regressor. Also setting max_depth to prevent the decision tree being too deep leads to overfitting and wasting time. And see how much time it takes and how accuracy it inprove.

[package](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [41]:
clf = OneVsRestClassifier(DecisionTreeRegressor(max_depth = 2))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.3774
Time: 232.54 sec


In [45]:
clf = OneVsRestClassifier(DecisionTreeRegressor(max_depth = 5))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.3768
Time: 255.86 sec


### Support vector machine(SVR)

Use Epsilon-Support Vector Regression and the implementation is based on libsvm.

Also try some kernel to see the results and time it takes.

[package](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

In [43]:
ker = ['linear', 'poly', 'rbf', 'sigmoid']

for item in ker:
    print 'kernel: {}'.format(item)
    clf = OneVsRestClassifier(SVR(kernel = item))
    eval_model(False, clf)

kernel: linear
Mean square error for 48 timeslot: 0.4756
Time: 248.61 sec
kernel: poly
Mean square error for 48 timeslot: 0.4800
Time: 261.04 sec
kernel: rbf
Mean square error for 48 timeslot: 0.4801
Time: 266.96 sec
kernel: sigmoid
Mean square error for 48 timeslot: 0.4622
Time: 245.75 sec


### Other


Regression based on k-nearest neighbors.

The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set.

In [38]:
clf = OneVsRestClassifier(KNeighborsRegressor(n_neighbors = 3))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.4309
Time: 255.25 sec


In [44]:
clf = OneVsRestClassifier(KNeighborsRegressor(n_neighbors = 4))
eval_model(False, clf)

Mean square error for 48 timeslot: 0.4516
Time: 267.37 sec


### Compare and Observation

- ARIMA
- Bayesian regression
- Decision tree regression
- Support vector machine(SVR)
- Regression k-nearest neighbors

Try your own method to solve this prediction problem,and give a result
and some explanation.
Some hint:
You can try other model,use diffenent parameters, or
consider different features,like day of week, other nearby stations’
inflow or the transition probability.